In [1]:
%reload_ext nb_black


<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn import datasets, metrics
from sqlalchemy import create_engine
import warnings

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


<IPython.core.display.Javascript object>

In [3]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "heartdisease"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
df = pd.read_sql_query("select * from heartdisease", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

<IPython.core.display.Javascript object>

In [4]:
# Define the features and the outcome
X = df.iloc[:, :13]
y = df.iloc[:, 13]

# Replace missing values (marked by `?`) with a `0`
X = X.replace(to_replace="?", value=0)

# Binarize `y` so that `1` means heart disease diagnosis and `0` means no diagnosis
y = np.where(y > 0, 0, 1)

<IPython.core.display.Javascript object>

In [6]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

<IPython.core.display.Javascript object>

In [9]:
gmm = GaussianMixture(n_components=2)
clusters = gmm.fit_predict(X_std)

<IPython.core.display.Javascript object>

In [11]:
ari = metrics.adjusted_rand_score(y, clusters)
sil = metrics.silhouette_score(X_std, clusters, metric="euclidean")
print(f"ari: {ari}, Silhouette: {sil}")

ari: 0.4207322145049338, Silhouette: 0.16118591340148433


<IPython.core.display.Javascript object>

This performs slightly worse than kmeans on silhouette but better on ARI. Could be a matter of tuning hyperparameters. Makes sense considering it uses kmeans to initialize.

In [12]:
cov_types = ["full", "tied", "diag", "spherical"]
for cov in cov_types:
    gmm = GaussianMixture(n_components=2, covariance_type=cov)
    clusters = gmm.fit_predict(X_std)
    ari = metrics.adjusted_rand_score(y, clusters)
    sil = metrics.silhouette_score(X_std, clusters, metric="euclidean")
    print(f"Type: {cov} ari: {ari}, Silhouette: {sil}")

Type: full ari: 0.4207322145049338, Silhouette: 0.16118591340148433
Type: tied ari: 0.4558104186161976, Silhouette: 0.1671559472293439
Type: diag ari: 0.37878842718089933, Silhouette: 0.15836933745078682
Type: spherical ari: 0.20765243525722465, Silhouette: 0.12468753110276873


<IPython.core.display.Javascript object>

Covariance of 'tied' seems to be the best on both ARI and Silhouette.